# Set up, data, dataframe creation

In [46]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

#Set working directory 
#os.chdir('/Users/Davide/Documents/University/RA/text_analysis')
os.getcwd()


'/Users/Davide/Documents/University/RA/text_analysis/sentiment_analysis-master'

In [47]:
headlines_df = pd.read_csv('express.csv', index_col = 'date')
bbc_df = pd.read_csv('News_dataset.csv', sep=';')
kaggle_df = pd.read_json (r'/Users/Davide/Documents/University/RA/text_analysis/sentiment_analysis-master/News_Category_Dataset_v2.json', lines = True)

In [48]:
headlines_df

,headline,newspaper
date,,
1-3-2015,Prince Andrew 'frozen out by Charles over dama...,Express
1-3-2015,EXCLUSIVE: Migrants to put Britain's populatio...,Express
1-3-2015,EXCLUSIVE: Jihadi John exposed by web error: K...,Express
1-3-2015,Missing Becky Watts: Two arrested as family ad...,Express
1-3-2015,We can all benefit from a positive approach to...,Express
...,...,...
31-3-2017,Man United player: I was worried after manager...,Express
31-3-2017,Mark Lawrenson: My biggest worry about Liverpool,Express
31-3-2017,La Liga ace reveals messages received from Man...,Express


In [49]:
bbc_df

,File_Name,Content,Category,Complete_Filename
0,001.txt,Ad sales boost Time Warner profit\n\nQuarterly...,business,001.txt-business
1,002.txt,Dollar gains on Greenspan speech\n\nThe dollar...,business,002.txt-business
2,003.txt,Yukos unit buyer faces loan claim\n\nThe owner...,business,003.txt-business
3,004.txt,High fuel prices hit BA's profits\n\nBritish A...,business,004.txt-business
4,005.txt,Pernod takeover talk lifts Domecq\n\nShares in...,business,005.txt-business
...,...,...,...,...
2220,397.txt,BT program to beat dialler scams\n\nBT is intr...,tech,397.txt-tech
2221,398.txt,Spam e-mails tempt net shoppers\n\nComputer us...,tech,398.txt-tech
2222,399.txt,Be careful how you code\n\nA new European dire...,tech,399.txt-tech
2223,400.txt,US cyber security chief resigns\n\nThe man mak...,tech,400.txt-tech


In [50]:
#Slice the df to have only Green and environment articles
green_df = kaggle_df[kaggle_df.category == 'GREEN']
env_df = kaggle_df[kaggle_df.category == 'ENVIRONMENT']


# Extracting Keywords

In [51]:
# Cleaning
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
nltk.download('wordnet')


def cleaning(df, column_str):
    '''function to clean a text within a column of a dataframe. The column name has to be a string'''
    #Special character
    df[column_str] = df[column_str].str.replace("\r", " ")
    df[column_str] = df[column_str].str.replace("\n", " ")
    df[column_str] = df[column_str].str.replace("    ", " ")
    df[column_str] = df[column_str].str.replace('"', '')

    #Lowercase
    df[column_str] = df[column_str].str.lower()

    #Punctuation
    punctuation_signs = list("?:!.,;-")

    for punct_sign in punctuation_signs:
        df[column_str] = df[column_str].str.replace(punct_sign, '')

    #Possessive pronouns
    df[column_str] = df[column_str].str.replace("'s", "")
    df[column_str] = df[column_str].str.replace("'", "")

    #Lemmatization
    # Saving the lemmatizer into an object
    wordnet_lemmatizer = WordNetLemmatizer()

    nrows = len(df)
    lemmatized_text_list = []

    for row in range(0, nrows):

        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        text = df.iloc[row][column_str]
        text_words = text.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)

    df[column_str] = lemmatized_text_list

    #Stopwords
    # Downloading the stop words list
    nltk.download('stopwords')
    # Loading the stop words in english
    stop_words = list(stopwords.words('english'))

    df[column_str] = df[column_str]

    for stop_word in stop_words:

        regex_stopword = r"\b" + stop_word + r"\b"
        df[column_str] = df[column_str].str.replace(regex_stopword, '')

    return df

[nltk_data] Downloading package punkt to /Users/Davide/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Davide/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
green_df = cleaning(green_df, 'headline')
bbc_df = cleaning(bbc_df, 'Content')
env_df = cleaning(env_df, 'headline')

/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/Davide/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [53]:
biz_df = bbc_df[bbc_df.Category == 'business']
pol_df = bbc_df[bbc_df.Category == 'politics']

In [73]:
#Most commonly used words extractor
def keywords_extractor(df, headline_str, n):
    '''Return the most commonly used words from a dataframe with a column containing text.
    Arguments are dataframe, the name of the column in string format, and the number n of keywords needed'''
    df_words = Counter()
    df[headline_str].str.split().apply(df_words.update)
    df_words = df_words.most_common(20)
    df_words_lst = []
    for tup in df_words:
        df_words_lst.append(tup[0])
    return df_words_lst

#Keywords list creation
biz_keywords = keywords_extractor(biz_df, 'Content', n = 20)
pol_keywords = keywords_extractor(pol_df, 'Content', n = 20)
green_keywords = keywords_extractor(green_df, 'headline', n = 20)
env_keywords = keywords_extractor(env_df, 'headline', n = 20)

In [74]:
print(biz_keywords)

['say', 'us', 'year', 'company', 'mr', 'firm', 'market', 'would', 'bank', 'rise', 'also', 'new', 'price', 'share', 'growth', 'last', 'economy', 'make', 'government', 'sales']


In [75]:
print(pol_keywords)

['say', 'mr', 'would', 'labour', 'government', 'party', 'people', 'blair', 'election', 'minister', 'plan', 'make', 'also', 'new', 'tell', 'could', 'brown', 'go', 'tax', 'lord']


In [111]:
#Combining green and env keywords
climate_keywords = []
climate_keywords.extend(green_keywords)
climate_keywords.extend(env_keywords)

#Eliminating duplicates
climate_keywords = list(set(climate_keywords))

#Removing not useful keywords
climate_keywords.remove('(photos)')
climate_keywords.remove('(video)')

print(climate_keywords)

['save', 'picture', 'find', 'world', 'trump', 'make', 'extreme', 'animal', 'weather', 'say', 'take', 'water', 'week', 'dog', 'climate', 'watch', 'baby', 'new', 'kill', 'photos', '2012', 'oil', 'get', 'us', 'green', 'change', 'hurricane', 'day', 'energy', 'california', 'could']


# Classification

In [ ]:
british_keywords = ["brexit", "european union", " eu ", "british", "british identity", "british passport", "british culture", "british heritage", "british goods", "british products","british manufacturing", "made in britain"]
climate_keywords = ["climate change", "paris agreement", "environment", "global warming", "unfccc"]
immigration_keywords = [""]

headlines_df['classification'] = ''
classification_lst = []


def headline_classifier(string, keyword_lst_class_1, keyword_lst_class_2, name_class_1 = 'class_1', name_class_2 = 'class_2'):
    '''Classifies a list of strings according to keyword lists for class_1 and class_2'''
    string = string.lower()
    count_1 = 0
    count_2 = 0
    for keyword in keyword_lst_class_1:
        if keyword in string:
            count_1 =+ 1
    for keyword in keyword_lst_class_2:

        if keyword in string:
            count_2 =+ 1
    
    if count_1 > 0 and count_2 > 0:
        return 'both'
    elif count_1 > 0:
        return name_class_1
    elif count_2 > 0:
        return name_class_2
    else:
        return 'none'

In [ ]:
n_row, n_col = headlines_df.shape
headlines_lst = list(headlines_df['headline'].values)

#Headline classification
classification_lst = []
for i in range(n_row):
    headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
    classification_lst.append(headline_classifier(headline_i, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))

headlines_df['classification'] = classification_lst
headlines_df.head(50)
headlines_df.classification.value_counts()

In [ ]:
#Testing
#To do find more elegant way to deal with eu than " eu "
test_lst = []

test_headline = headlines_df.iloc[43]['headline']     
test_lst.append(headline_classifier(test_headline, british_keywords, climate_keywords, name_class_1 = 'Brexit', name_class_2 = 'Climate'))              
test_lst

test_lst

# Sentiment Analysis

In [ ]:
#Empty list to add the polarity score
polarity_lst = []
subjectivity_lst = []

#Headline sentiment
for i in range(n_row):
    headline_i = str(headlines_lst[i]) #Include this becasue of potential nan values
    blob_headline_i = TextBlob(headline_i) #transforming string into textblob
    polarity_lst.append(blob_headline_i.sentiment.polarity)
    subjectivity_lst.append(blob_headline_i.sentiment.subjectivity)

#Adding polarity and subjectivity scores to the headlines dataframe
headlines_df['polarity'] = polarity_lst
headlines_df['subjectivity'] = subjectivity_lst
headlines_df

In [ ]:
#Creating dataframe for each class
brexit_df = headlines_df[headlines_df.classification == 'Brexit']
climate_df = headlines_df[headlines_df.classification == 'Climate']
both_df = headlines_df[headlines_df.classification == 'both']

#
results_df = pd.DataFrame(index = ['polarity', 'subjectivity'], columns = ['Brexit', 'Climate', 'Both'])
results_df['Brexit'] = brexit_df.mean(axis = 0, numeric_only = True)
results_df['Climate'] = climate_df.mean(axis = 0, numeric_only = True)
results_df['Both'] = both_df.mean(axis = 0, numeric_only = True)

results_df